# Importando as bibliotecas:

In [1]:
# Importando as bibliotecas necessárias:
import numpy as np
import tensorflow as tf
import cv2
import os
import time
from google.colab import drive

# Carregando os modelos da memória:

In [2]:
# Conectando ao Google Drive:
drive.mount('/content/drive')
caminho =  '/content/drive/MyDrive/yolo_pesos/tf_lite_model_f32.tflite'

# Carregar o Modelo TFLite da Memória
with open(caminho, 'rb') as f:
    tflite_model = f.read()

# Instanciar um Interpretador
try:
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()
except Exception as e:
    print("Erro ao carregar o modelo:", e)
else:
    print("Modelo carregado com sucesso.\n")

# Preparar as Entradas:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

# Exibindo informações das entradas:
print(input_details)
print(output_details)
print(input_shape)

Mounted at /content/drive
Modelo carregado com sucesso.

[{'name': 'serving_default_images:0', 'index': 0, 'shape': array([  1,   3, 320, 320], dtype=int32), 'shape_signature': array([  1,   3, 320, 320], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 858, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1, -1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[  1   3 320 320]


In [4]:
print(input_details[0]['index'])
print(output_details[0]['index'])

0
858


# Funções utilizadas:

In [ ]:
# Função para aplicar resize e normalizar a imagem para a entrada do:
def image_process(imagem, escala_x, escala_y):
  # Redimensionar a imagem para as dimensões esperadas pelo modelo
  resized_image = cv2.resize(imagem, (escala_x, escala_y))

  # Converter de BGR para RGB (OpenCV carrega imagens como BGR por padrão)
  new_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

  # Converter para ponto flutuante e normalizar
  image_norm = new_image.astype(np.float32) / 255.0

  # Transpor as dimensões para corresponder à forma esperada pelo modelo
  image_norm = image_norm.transpose(2, 0, 1)

  # Adicionar uma dimensão extra para corresponder à forma esperada pelo modelo
  image_norm = np.expand_dims(image_norm, axis=0)

  return resized_image, image_norm

In [ ]:
# Função para realizar inferência com o modelo tflite:
def inferencia(image_norm, input_details, output_details, interpreter):
  # Definir os dados de entrada do modelo
  interpreter.set_tensor(input_details[0]['index'], image_norm)

  start = time.time()
  # Realizar a inferência
  interpreter.invoke()
  end = time.time()
  inference_time = end-start

  # Obter os resultados da inferência
  output_data = interpreter.get_tensor(output_details[0]['index'])

  # Filtrar as linhas onde a probabilidade é maior que 0.5:
  filtered_output_data = [detection for detection in output_data if detection[6] > 0.5]

  return filtered_output_data, inference_time

In [ ]:
def calcular_iou(caixa1, caixa2):
    # Extrair coordenadas das caixas delimitadoras
    x1_c1, y1_c1, largura_c1, altura_c1 = caixa1
    x1_c2, y1_c2, largura_c2, altura_c2 = caixa2

    # Calcular coordenadas dos vértices opostos
    x2_c1 = x1_c1 + largura_c1
    y2_c1 = y1_c1 + altura_c1
    x2_c2 = x1_c2 + largura_c2
    y2_c2 = y1_c2 + altura_c2

    # Calcular coordenadas da interseção
    x1_intersecao = max(x1_c1, x1_c2)
    y1_intersecao = max(y1_c1, y1_c2)
    x2_intersecao = min(x2_c1, x2_c2)
    y2_intersecao = min(y2_c1, y2_c2)

    # Calcular área de interseção
    area_intersecao = max(0, x2_intersecao - x1_intersecao + 1) * max(0, y2_intersecao - y1_intersecao + 1)

    # Calcular áreas das caixas delimitadoras
    area_caixa1 = (x2_c1 - x1_c1 + 1) * (y2_c1 - y1_c1 + 1)
    area_caixa2 = (x2_c2 - x1_c2 + 1) * (y2_c2 - y1_c2 + 1)

    # Calcular área da união
    area_uniao = area_caixa1 + area_caixa2 - area_intersecao

    # Calcular IoU
    iou = area_intersecao / area_uniao

    return iou

In [ ]:
def teste_modelo(diretorio_images, diretorio_labels, interpreter, input_details, output_details, input_shape):
  # Listar todos os arquivos no diretório
  images = os.listdir(diretorio_images)
  labels = os.listdir(diretorio_labels)

  escala_x = input_shape[2]
  escala_y = input_shape[3]

  # Criando nossos contadores de instâncias:
  true_positives = 0
  wrong_detections = 0
  false_negatives = 0
  false_positives = 0

  tempos_de_inferencia = []

  # Verificar se o número de imagens e rótulos é o mesmo
  if len(images) != len(labels):
      print("O número de imagens e rótulos não corresponde.")
  else:
      # Loop sobre as imagens e rótulos ao mesmo tempo
      for imagem_nome, label_nome in zip(images, labels):
          # Caminhos completos para a imagem e o rótulo
          caminho_imagem = os.path.join(diretorio_images, imagem_nome)
          caminho_label = os.path.join(diretorio_labels, label_nome)

          # Verificar se ambos são arquivos
          if os.path.isfile(caminho_imagem) and os.path.isfile(caminho_label):
              # Carregar a imagem usando OpenCV
              imagem = cv2.imread(caminho_imagem)

              # Verificar se a imagem foi carregada corretamente
              if imagem is not None:
                  # Processando a imagem:
                  img, img_norm = image_process(imagem, escala_x, escala_y)

                  # Realizando a inferência:
                  output, tempo_decorrido = inferencia(img_norm, input_details, output_details, interpreter)
                  tempos_de_inferencia.append(tempo_decorrido)

                  # Listas de detecções e rotulos da imagem atual:
                  deteccoes = []
                  rotulos = []

                  for detection in output:
                      # Obter os valores das coordenadas da bounding box:
                      _, x1, y1, x2, y2, classe_pred, probabilidade_pred = detection

                      # Calcular as dimensões da bounding box:
                      bbox_largura = x2 - x1
                      bbox_altura = y2 - y1

                      # Salva a detecção atual na lista de detecções:
                      deteccoes.append([x1, y1, bbox_largura, bbox_altura, classe_pred, probabilidade_pred])

                  # Ler os rotulos do arquivo .txt:
                  with open(caminho_label, 'r') as file:
                    for linha in file:
                        label_content = linha.strip()  # Remover espaços em branco extras
                        valores = label_content.split() # Dividir o conteúdo em uma lista de valores

                        # Converter os valores para os tipos apropriados
                        classe = int(valores[0])
                        centro_x = int(escala_x * float(valores[1]))
                        centro_y = int(escala_y * float(valores[2]))
                        largura = int(escala_x * float(valores[3]))
                        altura = int(escala_y * float(valores[4]))
                        x = centro_x - (largura//2)
                        y = centro_y - (altura//2)

                        # Salva a label atual na lista de rotulos:
                        rotulos.append([x, y, largura, altura, classe])

                  # Cria cópias das listas:
                  temp_detections = deteccoes.copy()
                  temp_labels = rotulos.copy()

                  # Realizndo o matching das bbox e suas classificações:
                  for detection in deteccoes:
                    for label in rotulos:
                      iou = calcular_iou(detection[:4], label[:4])
                      if (iou > 0.5):
                        #verifica se a classe é igual:
                        if (detection[4] == label[4]):
                          true_positives += 1
                        else:
                          wrong_detections += 1
                        # Remove essas da lista pq são do mesmo objeto
                        if detection in temp_detections:
                          temp_detections.remove(detection)
                        if label in temp_labels:
                          temp_labels.remove(label)

                  # Sobrou os detectados de forma errada:
                  false_positives += len(temp_detections)
                  false_negatives += len(temp_labels)

              else:
                  print(f"Não foi possível carregar a imagem: {caminho_imagem}")
          else:
              print(f"Arquivos ausentes para a imagem {imagem_nome} ou rótulo {label_nome}.")

  # Calculando as métricas do nosso teste:
  num_imgs = len(images)
  wrong_detections += false_positives + false_negatives
  tempos_de_inferencia_medio = np.mean(tempos_de_inferencia)
  acuracia = true_positives/(true_positives + wrong_detections)
  precision = true_positives / (true_positives + false_positives)
  recall = true_positives / (true_positives + false_negatives)
  f1_score = 2 * (precision * recall) / (precision + recall)

  # Exibindo as métricas e detalhes do teste:
  print("Metricas do teste")
  print(f"Imagens inferidas: {num_imgs}")
  print(f"Tempo de inferência médio: {tempos_de_inferencia_medio}")
  print(f"Detecções corretas: {true_positives}")
  print(f"Detecções erradas: {wrong_detections}")
  print(f"Objetos não detectados: {false_negatives}")
  print(f"Falsos objetos detectados: {false_positives}")
  print(f"Acurácia: {acuracia}")
  print(f"Precisão: {precision}")
  print(f"Recall: {recall}")
  print(f"Pontuação F1: {f1_score}")

# Realizando o teste:

In [ ]:
# Diretório contendo as imagens
diretorio_images = '/content/drive/MyDrive/yolo_pesos/test_descompactado/test/images'
diretorio_labels = '/content/drive/MyDrive/yolo_pesos/test_descompactado/test/labels'

# Testando:
teste_modelo(diretorio_images, diretorio_labels, interpreter, input_details, output_details, input_shape)

Metricas do teste
Imagens inferidas: 205
Tempo de inferência médio: 0.9482743995945628
Detecções corretas: 146
Detecções erradas: 85
Objetos não detectados: 62
Falsos objetos detectados: 13
Acurácia: 0.6320346320346321
Precisão: 0.9182389937106918
Recall: 0.7019230769230769
Pontuação F1: 0.7956403269754768
